# Orderbook with Heaps - Student Template

## Goal
Build an efficient orderbook using **heaps** for O(log P) best price lookups.

## Key Concept: Negative Prices for Max Heap
Python's `heapq` only does **min heaps** (smallest at top). For bids we need max heap (highest at top).

**Solution:** Store negative prices!
- Bid prices: $100, $99, $98
- Store as: -100, -99, -98
- Min heap puts -100 at top → negate back: $100 ✓

## Data Structures
```
bids = {price: deque([orders])}  # Price → queue of orders
asks = {price: deque([orders])}
bid_heap = [-100, -99, -98]      # Negative for max heap!
ask_heap = [101, 102, 103]       # Positive for min heap
orders = {order_id: Order}       # Fast lookup
```

## Setup (Run this first)

In [ ]:
from collections import deque
import heapq
from enum import Enum
from typing import List, Tuple, Optional, Dict

class Side(Enum):
    BUY = "BUY"
    SELL = "SELL"

class OrderStatus(Enum):
    OPEN = "OPEN"
    PARTIALLY_FILLED = "PARTIALLY_FILLED"
    FILLED = "FILLED"
    CANCELLED = "CANCELLED"

class Order:
    def __init__(self, order_id: str, side: Side, price: float, quantity: int):
        self.order_id = order_id
        self.side = side
        self.price = price
        self.quantity = quantity
        self.filled_quantity = 0
        self.status = OrderStatus.OPEN
    
    def remaining_quantity(self) -> int:
        return self.quantity - self.filled_quantity
    
    def __repr__(self):
        return f"Order({self.order_id}, {self.side.value}, ${self.price}, qty={self.quantity}, filled={self.filled_quantity}, {self.status.value})"

print("✓ Setup complete!")

## OrderBook Initialization

In [ ]:
class OrderBook:
    def __init__(self):
        # Price level: price → deque of orders
        self.bids: Dict[float, deque] = {}
        self.asks: Dict[float, deque] = {}
        
        # Heaps for fast best price lookup
        self.bid_heap: List[float] = []  # Store negative prices!
        self.ask_heap: List[float] = []  # Store positive prices
        
        # Order lookup
        self.orders: Dict[str, Order] = {}

print("✓ OrderBook class created!")

## TODO 1: Submit Order (~3 minutes)

**Instructions:**
1. BUY side is done as example
2. Complete the SELL side using the same pattern
3. Key difference: For asks, use positive price in heap

**Remember:**
- Bids: `heapq.heappush(self.bid_heap, -price)` (negative!)
- Asks: `heapq.heappush(self.ask_heap, price)` (positive!)

In [ ]:
def submit_order(self, order_id: str, side: Side, price: float, quantity: int) -> bool:
    # Check if order already exists
    if order_id in self.orders:
        return False
    
    # Create order
    order = Order(order_id, side, price, quantity)
    self.orders[order_id] = order
    
    if side == Side.BUY:
        # Add to bids
        if price not in self.bids:
            self.bids[price] = deque()
            heapq.heappush(self.bid_heap, -price)  # NEGATIVE for max heap!
        self.bids[price].append(order)
    else:
        # TODO: Add to asks (similar to bids but use positive price)
        # YOUR CODE HERE:
        # 1. Check if price not in self.asks, create deque
        # 2. Push POSITIVE price to self.ask_heap
        # 3. Append order to self.asks[price]
        pass
    
    return True

OrderBook.submit_order = submit_order
print("✓ TODO 1: Complete the asks section!")

## TODO 2: Cancel Order (~4 minutes)

**Instructions:**
Make cancellation more realistic - actually remove from the queue!

**Steps:**
1. Look up order
2. Check if already cancelled/filled
3. Mark as cancelled
4. Remove from the deque (use `queue.remove(order)`)
5. If queue is empty, delete price from dict

**Note:** We don't remove from heap (inefficient). Stale prices are cleaned up in get_best_bid/ask.

In [ ]:
def cancel_order(self, order_id: str) -> bool:
    # Look up order
    if order_id not in self.orders:
        return False
    
    order = self.orders[order_id]
    
    # Check if can be cancelled
    if order.status in [OrderStatus.FILLED, OrderStatus.CANCELLED]:
        return False
    
    # TODO: Mark as cancelled
    order.status = OrderStatus.CANCELLED
    
    # TODO: Remove from price level queue
    # Get the correct price_levels dict (bids or asks)
    price_levels = self.bids if order.side == Side.BUY else self.asks
    
    if order.price in price_levels:
        queue = price_levels[order.price]
        
        # YOUR CODE HERE:
        # 1. Remove order from queue using: queue.remove(order)
        # 2. If queue is now empty (len(queue) == 0), delete the price:
        #    del price_levels[order.price]
        try:
            pass  # Replace this with your code
        except ValueError:
            pass  # Order not in queue
    
    return True

OrderBook.cancel_order = cancel_order
print("✓ TODO 2: Remove order from queue and cleanup!")

## Get Order Status (Already done!)

In [ ]:
def get_order_status(self, order_id: str) -> Optional[Order]:
    """Look up order by ID - O(1) dictionary lookup."""
    return self.orders.get(order_id)

OrderBook.get_order_status = get_order_status
print("✓ Get order status complete!")

## TODO 3: Get Best Bid (~3 minutes)

**Instructions:**
Get highest bid price using the heap.

**Key points:**
- Heap may have stale prices (from cancelled orders)
- Clean them up: loop and check if price has valid orders
- Remember: bid_heap stores NEGATIVE prices!

**Algorithm:**
1. While bid_heap not empty:
2. Get price from top: `price = -self.bid_heap[0]` (negate!)
3. If price in self.bids and has orders → return price
4. Else pop stale price and continue

In [ ]:
def get_best_bid(self) -> Optional[float]:
    """Get highest bid price - O(1) amortized."""
    while self.bid_heap:
        # TODO: Get price from heap (remember to negate!)
        price = -self.bid_heap[0]  # Negate to get actual price
        
        # TODO: Check if price is valid (has orders)
        # YOUR CODE HERE:
        # If price in self.bids and len(self.bids[price]) > 0:
        #     return price
        # Else:
        #     heapq.heappop(self.bid_heap)  # Remove stale price
        #     if price in self.bids: del self.bids[price]
        pass
    
    return None

OrderBook.get_best_bid = get_best_bid
print("✓ TODO 3: Clean up stale prices and return best bid!")

## TODO 4: Get Best Ask (~2 minutes)

**Instructions:**
Same as get_best_bid but for asks.

**Key difference:** ask_heap uses POSITIVE prices (no negation needed!)

In [ ]:
def get_best_ask(self) -> Optional[float]:
    """Get lowest ask price - O(1) amortized."""
    while self.ask_heap:
        # TODO: Get price from heap (no negation for asks!)
        price = self.ask_heap[0]  # Already positive
        
        # YOUR CODE HERE: Same logic as get_best_bid
        # Check if valid, return or clean up
        pass
    
    return None

OrderBook.get_best_ask = get_best_ask
print("✓ TODO 4: Implement get_best_ask!")

## TODO 5: Match Order (~5 minutes)

**Instructions:**
Most of the structure is provided. Fill in the SELL matching logic.

**Algorithm:**
1. Submit incoming order
2. Determine opposite side (bids for SELL, asks for BUY)
3. Get best opposite price
4. Check if matchable
5. Match against orders at that price level
6. Update filled quantities and statuses
7. Record trades

In [ ]:
def match_order(self, order_id: str, side: Side, price: float, quantity: int) -> List[Tuple[str, str, float, int]]:
    """Match order against opposite side."""
    trades = []
    
    # Submit order
    if not self.submit_order(order_id, side, price, quantity):
        return trades
    
    incoming = self.orders[order_id]
    
    if side == Side.BUY:
        # BUY order: match against asks
        best_ask = self.get_best_ask()
        if best_ask is None or best_ask > price:
            return trades  # No match
        
        ask_orders = self.asks[best_ask]
        
        for resting in list(ask_orders):  # Use list() to avoid modification issues
            if resting.status != OrderStatus.OPEN:
                continue
            
            # Calculate trade quantity
            trade_qty = min(incoming.remaining_quantity(), resting.remaining_quantity())
            
            # Update quantities
            incoming.filled_quantity += trade_qty
            resting.filled_quantity += trade_qty
            
            # Update statuses
            if incoming.remaining_quantity() == 0:
                incoming.status = OrderStatus.FILLED
            elif incoming.filled_quantity > 0:
                incoming.status = OrderStatus.PARTIALLY_FILLED
                
            if resting.remaining_quantity() == 0:
                resting.status = OrderStatus.FILLED
            elif resting.filled_quantity > 0:
                resting.status = OrderStatus.PARTIALLY_FILLED
            
            # Record trade (buyer, seller, price, qty)
            trades.append((order_id, resting.order_id, best_ask, trade_qty))
            
            if incoming.remaining_quantity() == 0:
                break
    
    else:  # Side.SELL
        # TODO: Implement SELL order matching
        # YOUR CODE HERE:
        # 1. Get best_bid using self.get_best_bid()
        # 2. Check if best_bid is None or best_bid < price → return trades
        # 3. Get bid_orders = self.bids[best_bid]
        # 4. Loop through bid_orders (use list() wrapper)
        # 5. Calculate trade_qty, update quantities and statuses
        # 6. Record trade as: (resting.order_id, order_id, best_bid, trade_qty)
        #    Note: buyer is resting order, seller is incoming order!
        pass
    
    return trades

OrderBook.match_order = match_order
print("✓ TODO 5: Complete the SELL matching logic!")

## Pretty Print (Bonus - already done)

In [ ]:
def __repr__(self):
    lines = ["=" * 60, "ORDERBOOK", "=" * 60]
    
    # Show asks (high to low)
    for price in sorted(self.asks.keys(), reverse=True):
        total = sum(o.remaining_quantity() for o in self.asks[price] if o.status == OrderStatus.OPEN)
        if total > 0:
            lines.append(f"ASK: ${price:>8.2f} | Qty: {total}")
    
    lines.append("-" * 60)
    
    # Show bids (high to low)
    for price in sorted(self.bids.keys(), reverse=True):
        total = sum(o.remaining_quantity() for o in self.bids[price] if o.status == OrderStatus.OPEN)
        if total > 0:
            lines.append(f"BID: ${price:>8.2f} | Qty: {total}")
    
    lines.append("=" * 60)
    best_bid = self.get_best_bid()
    best_ask = self.get_best_ask()
    if best_bid and best_ask:
        lines.append(f"Spread: ${best_ask - best_bid:.2f}")
    return "\n".join(lines)

OrderBook.__repr__ = __repr__
print("✓ Pretty print ready!")

## Test Your Implementation!

In [ ]:
# Test 1: Submit orders
print("Test 1: Submit orders")
ob = OrderBook()

ob.submit_order("B1", Side.BUY, 100.0, 50)
ob.submit_order("B2", Side.BUY, 99.0, 30)
ob.submit_order("A1", Side.SELL, 101.0, 40)
ob.submit_order("A2", Side.SELL, 102.0, 25)

print(f"Best Bid: ${ob.get_best_bid()}")
print(f"Best Ask: ${ob.get_best_ask()}")
print(ob)

In [ ]:
# Test 2: Order status
print("Test 2: Order status")
order = ob.get_order_status("B1")
print(order)

In [ ]:
# Test 3: Cancel order
print("Test 3: Cancel order B2")
ob.cancel_order("B2")
print(f"Best Bid after cancel: ${ob.get_best_bid()}")
print(ob)

In [ ]:
# Test 4: Match sell order
print("Test 4: Match sell order at $99")
trades = ob.match_order("S1", Side.SELL, 99.0, 30)
print(f"Trades: {len(trades)}")
for buyer, seller, price, qty in trades:
    print(f"  {buyer} bought {qty} from {seller} at ${price}")
print(ob)

In [ ]:
# Test 5: Match buy order
print("Test 5: Match buy order at $105")
trades = ob.match_order("B3", Side.BUY, 105.0, 60)
print(f"Trades: {len(trades)}")
for buyer, seller, price, qty in trades:
    print(f"  {buyer} bought {qty} from {seller} at ${price}")
print(ob)

## Summary

You've implemented an efficient orderbook with:
- ✓ Heaps for O(log P) best price lookups
- ✓ Negative prices for max heap behavior
- ✓ Proper order cancellation with queue removal
- ✓ Order matching with partial fills
- ✓ Stale price cleanup

**Time Complexities:**
- Submit: O(log P)
- Cancel: O(M)
- Get Best: O(1) amortized
- Match: O(M) per price level